# Experiments


Sweep qubits (2–4), depth, shots, noise.

Log results to CSV.

Include IBM device run if possible (small circuit).

In [1]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import itertools
import joblib
import time
# from qiskit import IBMProvider


In [ ]:
params = np.load("params/trained_params.npy")      #trained parameters
 
#trained models
q_lg_clf = joblib.load("models/quantum_logreg.pkl")
q_svm_clf = joblib.load("models/quantum_svm.pkl")   


## Sweep Qubits (Circuit Hyperparameters)

Looping over a range of circuit hyperparamters (number of qubits, depth (# of layers in var. ansatz), shot (# of times qcircuit is sampled), noise (whether to stimulate device noise - will run on IBM)). 

- More qubits would mean a richer feature space but more noise/cost. 
- More depth/layers would mean higher expressivity but is more difficult to train
- More shots means lower measurement noise but a longer runtime
- More noise means noise realism tradeoff


In [ ]:
n_qubit_list = [2,3,4]
depth_list = [1,2]
shot_list= [None, 1024]      #none means no sampling - analytic
noise_list = [False, True]    #true = use simulator 

In [ ]:
for n_qubits, depth, shots, noise in itertools.product(n_qubit_list, depth_list, shot_list, noise_list):
    start = time.time()

    # Device setup
    if noise:
        dev = qml.device("default.mixed", wires=n_qubits, shots=shots)
    else:
        dev = qml.device("default.qubit", wires=n_qubits, shots=shots)

    @qml.qnode(dev)
    def qnode_measure(x, params):
        zz_feature_map(x, wires=range(n_qubits), reps=depth)
        var_ansatz(params, n_qubits)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    def get_quantum_features(X_mat, params):
        return np.array([qnode_measure(x[:n_qubits], params) for x in X_mat])

    # Initialize parameters
    params = np.random.randn(depth, n_qubits, 3)

    # Run training and testing
    Xq_train = get_quantum_features(X_train, params)
    Xq_test = get_quantum_features(X_test, params)

    clf = LogisticRegression(max_iter=500)
    clf.fit(Xq_train, y_train)
    acc = clf.score(Xq_test, y_test)

    # Log results
    results.append({
        "n_qubits": n_qubits,
        "depth": depth,
        "shots": shots if shots else "analytic",
        "noise": noise,
        "accuracy": acc,
        "runtime_sec": round(time.time() - start, 2)
    })

    print(f"Run ({n_qubits}q, depth={depth}, shots={shots}, noise={noise}) → acc={acc:.3f}")






## Log Results to CSV

In [ ]:
df = pd.DataFrame(results)
df.to_csv("results/quantum_sweep_results.csv", index=False)
print("\nSaved to results/quantum_sweep_results.csv")

## Run on IBM Hardware 

In [ ]:
provider = IBMProvider()
backend = provider.get_backend("ibmq_qasm_simulator")  
dev = qml.device("qiskit.ibmq", wires=2, backend=backend, shots=1024)

